In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dot, Flatten, Dense

In [2]:
# Exemplo de corpus de texto
corpus = [
    'Quais foram os 5 maiores fornecedores de janeiro a março de 2024?',
    'Qual o total e percentual de pagamentos manuais ocorridos em janeiro de 2024?,'
    'Quanto tenho a pagar para o fornecedor Google?',
    'Quanto tenho a pagar para os fornecedores X, Y, Z?',
    'Quanto tenho a pagar para fornecedores do grupo 4000 (Jurídico)?'
    'Quanto paguei ontem para a empresa 1000?',
    'Qual o total que tenho a pagar para a empresa 1100 hoje?',
    'Pagamentos manuais ou pagamento manual (Z, E, B).',
    'Pagamentos (total, listagem de, etc.) efetuados após o vencimento, considerando-se calendário FEBRABAN, por empresa e intervalo de tempo.',
    'Percentual de pagamentos efetuados após o vencimento, considerando o total de vencimentos.',
    'Documentos vencidos, por empresa, por intervalo de tempo, por fornecedor, etc., considerando-se o calendário FEBRABAN.',
    'Relação de documentos (ou total) a vencer (considerando calendário FEBRABAN) em dado período de tempo.',
    'Qual o total de pagamentos efetuados após o vencimento pela empresa X no período de janeiro a março de 2024?',
    'Listagem de pagamentos efetuados após o vencimento para o fornecedor Y considerando calendário FEBRABAN.',
    'Percentual de documentos vencidos para o grupo 4000 no primeiro trimestre de 2024.',
    'Relação de documentos a vencer para o fornecedor Google no mês de fevereiro de 2024.',
    'Total de pagamentos vencidos por empresa 1000 no ano de 2023.',
    'Percentual de pagamentos manuais feitos para os fornecedores X, Y e Z em janeiro de 2024.',
    'Quais são os documentos a vencer para o grupo Jurídico na próxima semana?',
    'Qual o total de pagamentos efetuados em janeiro de 2024?',
    'Qual o percentual de pagamentos manuais feitos em fevereiro de 2024?',
    'Qual o percentual de pagamentos manuais feitos em fevereiro de 2024?',
    'Qual foi o valor total pago para o fornecedor Microsoft no mês de março de 2024?',
    'Quanto tenho a pagar para fornecedores do grupo 3000 (TI)?',
    'Quanto paguei anteontem para a empresa 2000?',
    'Qual o total que tenho a pagar para a empresa 2100 amanhã?',
    'Pagamentos manuais efetuados para (A, B, C) no primeiro trimestre de 2024.',
    'Quais foram os 10 maiores fornecedores de fevereiro a abril de 2024?',
    'Quanto foi pago em junho de 2024 para fornecedores do grupo 5000 (Financeiro)?'
    'Qual o valor total de pagamentos vencidos para a empresa 3100 em maio de 2024?.',
    'Qual é a lista de fornecedores do grupo 6000 (Marketing) no mês de julho de 2024?',
    'Quantos documentos venceram em agosto de 2024 para o fornecedor Amazon?',
    'Relação de documentos a vencer em setembro de 2024 considerando o calendário FEBRABAN.',
    'Qual é a relação de documentos a vencer para a empresa 9000 no próximo semestre?',
    'Quantos pagamentos manuais foram feitos para o fornecedor Oracle no primeiro trimestre de 2023?',
    'Qual o total de documentos a vencer no mês de fevereiro de 2023 para o grupo 1200 (Comercial)?',
    'Qual o percentual de pagamentos efetuados após o vencimento para a empresa 8000 no quarto trimestre de 2024?',
    'Quanto foi pago para fornecedores do grupo 1100 (Administrativo) no último semestre?',
    'Relação de documentos vencidos para a empresa 7000 considerando o calendário FEBRABAN.',
    'Quanto tenho a pagar para fornecedores do grupo 1000 (TI) no próximo trimestre?',
    'Quais são os fornecedores do grupo 9000 (Logística) que têm documentos a vencer no próximo mês?',
    'Qual o valor total de documentos vencidos para o fornecedor IBM no ano de 2022?',
    'Quantos pagamentos manuais foram efetuados para a empresa 6000 no mês passado?',
    'Qual é o percentual de pagamentos efetuados manualmente para fornecedores do grupo 8000 (Operacional) em dezembro de 2024?',
    'Quanto foi pago para o fornecedor Facebook no ano de 2023?',
    'Qual o total de documentos a vencer em novembro de 2024 para a empresa 5000?',
    'Quais são os fornecedores que mais receberam pagamentos manuais em outubro de 2024?',
    'Percentual de documentos vencidos para o grupo 7000 (RH) no segundo trimestre de 2024.',
    'Qual o total de pagamentos efetuados após o vencimento pela empresa 4000 no período de abril a junho de 2024?',
    'Relação de documentos a vencer em setembro de 2024 considerando o calendário FEBRABAN.',
]
 
# Tokenização do texto
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
word2id = tokenizer.word_index
id2word = {v: k for k, v in word2id.items()}
vocab_size = len(word2id) + 1  # +1 para incluir o índice 0
print('palavra para id: ','\n',word2id)
print('id para palavra: ','\n',id2word)

palavra para id:  
 {'de': 1, 'o': 2, 'a': 3, 'para': 4, '2024': 5, 'pagamentos': 6, 'no': 7, 'qual': 8, 'total': 9, 'empresa': 10, 'documentos': 11, 'em': 12, 'fornecedores': 13, 'grupo': 14, 'quanto': 15, 'efetuados': 16, 'percentual': 17, 'manuais': 18, 'fornecedor': 19, 'vencer': 20, 'do': 21, 'considerando': 22, 'os': 23, 'tenho': 24, 'pagar': 25, 'calendário': 26, 'febraban': 27, 'vencidos': 28, 'após': 29, 'vencimento': 30, 'relação': 31, 'trimestre': 32, 'mês': 33, 'quais': 34, 'janeiro': 35, 'por': 36, 'fevereiro': 37, 'foram': 38, 'e': 39, 'que': 40, '2023': 41, 'feitos': 42, 'foi': 43, 'pago': 44, 'março': 45, 'x': 46, 'y': 47, 'z': 48, '4000': 49, '1000': 50, 'tempo': 51, 'período': 52, 'primeiro': 53, 'ano': 54, 'são': 55, 'valor': 56, 'é': 57, 'quantos': 58, 'próximo': 59, 'maiores': 60, 'google': 61, 'jurídico': 62, 'paguei': 63, '1100': 64, 'ou': 65, 'b': 66, 'listagem': 67, 'etc': 68, 'se': 69, 'intervalo': 70, 'pela': 71, 'ti': 72, 'abril': 73, 'junho': 74, '5000': 75

In [3]:
# Gerar pares de treinamento
def generate_training_data(sentences, window_size, vocab_size):
    pairs = []
    for sentence in sentences:
        encoded = tokenizer.texts_to_sequences([sentence])[0]
        for idx, word_id in enumerate(encoded):
            context_ids = encoded[max(0, idx - window_size):idx] + encoded[idx + 1:min(len(encoded), idx + window_size + 1)]
            for context_id in context_ids:
                pairs.append((word_id, context_id))
    return pairs

window_size = 2
pairs = generate_training_data(corpus, window_size, vocab_size)
pairs = np.array(pairs)
print(pairs[:5])

[[34 38]
 [34 23]
 [38 34]
 [38 23]
 [38 82]]


In [4]:
# Construção do modelo Skip-gram
embedding_dim = 128
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=1))
model.add(Flatten())
model.add(Dense(units=vocab_size, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 1, 128)            16768     
                                                                 
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 131)               16899     
                                                                 
Total params: 33667 (131.51 KB)
Trainable params: 33667 (131.51 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [5]:
# Preparação dos dados de entrada e saída
x_train = pairs[:, 0]
y_train = pairs[:, 1]

In [6]:
# Treinamento do modelo
model.fit(x_train, y_train, epochs=1000, verbose=True)

Epoch 1/1000
74/74 [==============================] - 0s 1ms/step - loss: 4.8250
Epoch 2/1000
74/74 [==============================] - 0s 835us/step - loss: 4.6427
Epoch 3/1000
74/74 [==============================] - 0s 836us/step - loss: 4.3201
Epoch 4/1000
74/74 [==============================] - 0s 843us/step - loss: 3.9645
Epoch 5/1000
74/74 [==============================] - 0s 783us/step - loss: 3.7264
Epoch 6/1000
74/74 [==============================] - 0s 709us/step - loss: 3.5561
Epoch 7/1000
74/74 [==============================] - 0s 710us/step - loss: 3.4207
Epoch 8/1000
74/74 [==============================] - 0s 740us/step - loss: 3.3100
Epoch 9/1000
74/74 [==============================] - 0s 716us/step - loss: 3.2176
Epoch 10/1000
74/74 [==============================] - 0s 799us/step - loss: 3.1417
Epoch 11/1000
74/74 [==============================] - 0s 786us/step - loss: 3.0781
Epoch 12/1000
74/74 [==============================] - 0s 825us/step - loss: 3.0209
Epo

In [7]:
# Obter embeddings
embeddings = model.layers[0].get_weights()[0]
print(f"Embedding da palavra 'fornecedores': ")
print(embeddings[word2id['fornecedores']])

Embedding da palavra 'fornecedores': 
[-0.04418514 -0.1719562   0.06151203  0.6851387  -0.06933375 -0.1429431
  0.2913295   0.6279667  -0.09101318 -0.06445476 -0.34210122 -0.04173389
  0.33130774 -0.1671617  -0.42530563 -0.34300035  0.04385028 -0.36328092
  0.31755358  0.37797806  0.36019158  0.2826452   0.04579625  0.36117882
  0.74896544  0.37237287 -0.34193766  0.03192267 -0.06908579  0.7814727
 -0.37975568 -0.35495207  0.4967248   0.07473761 -0.25579277 -0.33952102
  0.43027985 -0.20845209  0.2543148  -0.6166819  -0.10485866 -0.01748447
 -0.6231938  -0.16363454  0.26307762 -0.82291955  0.07394762 -0.12771155
 -0.34531444  0.77769685  0.0752359   0.22866291  0.4929254   0.18714927
 -0.04722292  0.40635985  0.04082983  0.06623816 -0.1933808  -0.07632983
 -0.2116977   0.5096948  -0.21485546 -0.6741889   0.36185288 -0.26166758
 -0.37349752  0.17283805 -0.10523226  0.03260375  0.08433946  0.66878396
  0.3145573   0.7956747   0.28371048 -0.31900266  0.1206705   0.25446337
 -0.24630393 -0

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

# Exemplo de corpus anotado para NER (simplificado)
texts = [
    'Quais foram os 5 maiores fornecedores de janeiro a março de 2024?', 
    'Qual o total e percentual de pagamentos manuais ocorridos em janeiro de 2024?', 
    'Quanto tenho a pagar para o fornecedor Google?', 
    'Quanto tenho a pagar para os fornecedores X, Y, Z?', 
    'Quanto tenho a pagar para fornecedores do grupo 4000 (Jurídico)?', 
    'Quanto paguei ontem para a empresa 1000?', 
    'Qual o total que tenho a pagar para a empresa 1100 hoje?', 
    'Pagamentos manuais ou pagamento manual (Z, E, B).', 
    'Pagamentos (total, listagem de, etc.) efetuados após o vencimento, considerando-se calendário FEBRABAN, por empresa e intervalo de tempo.', 
    'Percentual de pagamentos efetuados após o vencimento, considerando o total de vencimentos.', 
    'Documentos vencidos, por empresa, por intervalo de tempo, por fornecedor, etc., considerando-se o calendário FEBRABAN.', 
    'Relação de documentos (ou total) a vencer (considerando calendário FEBRABAN) em dado período de tempo.', 
    'Qual o total de pagamentos efetuados após o vencimento pela empresa X no período de janeiro a março de 2024?', 
    'Listagem de pagamentos efetuados após o vencimento para o fornecedor Y considerando calendário FEBRABAN.', 
    'Percentual de documentos vencidos para o grupo 4000 no primeiro trimestre de 2024.', 
    'Relação de documentos a vencer para o fornecedor Google no mês de fevereiro de 2024.', 
    'Total de pagamentos vencidos por empresa 1000 no ano de 2023.', 
    'Percentual de pagamentos manuais feitos para os fornecedores X, Y e Z em janeiro de 2024.', 
    'Quais são os documentos a vencer para o grupo Jurídico na próxima semana?', 
    'Qual o total de pagamentos efetuados em janeiro de 2024?', 
    'Qual o percentual de pagamentos manuais feitos em fevereiro de 2024?', 
    'Qual o percentual de pagamentos manuais feitos em fevereiro de 2024?', 
    'Qual foi o valor total pago para o fornecedor Microsoft no mês de março de 2024?', 
    'Quanto tenho a pagar para fornecedores do grupo 3000 (TI)?', 
    'Quanto paguei anteontem para a empresa 2000?', 
    'Qual o total que tenho a pagar para a empresa 2100 amanhã?', 
    'Pagamentos manuais efetuados para (A, B, C) no primeiro trimestre de 2024.', 
    'Quais foram os 10 maiores fornecedores de fevereiro a abril de 2024?', 
    'Quanto foi pago em junho de 2024 para fornecedores do grupo 5000 (Financeiro)?',
    'Qual o valor total de pagamentos vencidos para a empresa 3100 em maio de 2024?.', 
    'Qual é a lista de fornecedores do grupo 6000 (Marketing) no mês de julho de 2024?', 
    'Quantos documentos venceram em agosto de 2024 para o fornecedor Amazon?', 
    'Relação de documentos a vencer em setembro de 2024 considerando o calendário FEBRABAN.', 
    'Qual é a relação de documentos a vencer para a empresa 9000 no próximo semestre?', 
    'Quantos pagamentos manuais foram feitos para o fornecedor Oracle no primeiro trimestre de 2023?', 
    'Qual o total de documentos a vencer no mês de fevereiro de 2023 para o grupo 1200 (Comercial)?', 
    'Qual o percentual de pagamentos efetuados após o vencimento para a empresa 8000 no quarto trimestre de 2024?', 
    'Quanto foi pago para fornecedores do grupo 1100 (Administrativo) no último semestre?', 
    'Relação de documentos vencidos para a empresa 7000 considerando o calendário FEBRABAN.', 
    'Quanto tenho a pagar para fornecedores do grupo 1000 (TI) no próximo trimestre?', 
    'Quais são os fornecedores do grupo 9000 (Logística) que têm documentos a vencer no próximo mês?', 
    'Qual o valor total de documentos vencidos para o fornecedor IBM no ano de 2022?', 
    'Quantos pagamentos manuais foram efetuados para a empresa 6000 no mês passado?', 
    'Qual é o percentual de pagamentos efetuados manualmente para fornecedores do grupo 8000 (Operacional) em dezembro de 2024?',
    'Quanto foi pago para o fornecedor Facebook no ano de 2023?', 
    'Qual o total de documentos a vencer em novembro de 2024 para a empresa 5000?', 
    'Quais são os fornecedores que mais receberam pagamentos manuais em outubro de 2024?', 
    'Percentual de documentos vencidos para o grupo 7000 (RH) no segundo trimestre de 2024.', 
    'Qual o total de pagamentos efetuados após o vencimento pela empresa 4000 no período de abril a junho de 2024?', 
    'Relação de documentos a vencer em setembro de 2024 considerando o calendário FEBRABAN.'
]
ner_tags = [
    'O O O MIN_MAX MIN_MAX TABELA O INTERVALO INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O O O O O TABELA CONDICAO CONDICAO O INTERVALO INTERVALO INTERVALO?', 
    'O O O O O O O FORNECEDOR?', 
    'O O O O O O TABELA FORNECEDOR, FORNECEDOR, FORNECEDOR?', 
    'O O O O O TABELA O O ID O', 
    'O O O O O O ID?', 
    'O O O O O O O O O O ID INTERVALO?', 
    'TABELA CONDICAO CONDICAO O O (FORNECEDOR, FORNECEDOR, FORNECEDOR).', 
    'TABELA O O O O O CONDICAO CONDICAO, O O O O O O O O O O', 
    'O O TABELA O CONDICAO CONDICAO, O O O O O O', 
    'TABELA CONDICAO CONDICAO, O O O O O O O O O O O O', 
    'O O TABELA O O CONDICAO CONDICAO O O O O O O O O', 
    'O O O O TABELA O CONDICAO CONDICAO O O FORNECEDOR O O O INTERVALO INTERVALO INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O TABELA O CONDICAO CONDICAO O O O FORNECEDOR O O O O', 
    'O O TABELA CONDICAO O O O ID O INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O TABELA CONDICAO CONDICAO O O O FORNECEDOR O O O INTERVALO INTERVALO INTERVALO?', 
    'O O TABELA CONDICAO O ID INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O TABELA CONDICAO O O O TABELA FORNECEDOR, FORNECEDOR O FORNECEDOR O INTERVALO INTERVALO INTERVALO?', 
    'O O O TABELA CONDICAO CONDICAO O O O O O INTERVALO INTERVALO?', 
    'O O O O TABELA O O INTERVALO INTERVALO INTERVALO?', 
    'O O O O TABELA CONDICAO CONDICAO O INTERVALO INTERVALO INTERVALO?', 
    'O O O O TABELA CONDICAO CONDICAO O INTERVALO INTERVALO INTERVALO?', 
    'O O O O O O O O O FORNECEDOR O INTERVALO INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O O O O TABELA O O ID O', 
    'O O INTERVALO O O O ID?', 
    'O O O O O O O O O O ID INTERVALO?', 
    'TABELA CONDICAO CONDICAO O (FORNECEDOR, FORNECEDOR, FORNECEDOR) O INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O O MIN_MAX MIN_MAX TABELA O INTERVALO INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O O O INTERVALO INTERVALO INTERVALO O TABELA O O ID O', 
    'O O O O O TABELA CONDICAO O O O ID O INTERVALO INTERVALO INTERVALO?', 
    'O O O O O TABELA O O ID O O O O INTERVALO INTERVALO INTERVALO?', 
    'O TABELA CONDICAO O INTERVALO INTERVALO INTERVALO O O O FORNECEDOR?', 
    'O O TABELA CONDICAO CONDICAO O INTERVALO INTERVALO INTERVALO O O O O', 
    'O O O O O TABELA CONDICAO CONDICAO O O O ID O INTERVALO ?', 
    'O TABELA CONDICAO O O O O O FORNECEDOR O INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O O O TABELA CONDICAO CONDICAO O O O INTERVALO INTERVALO INTERVALO O O O ID O', 
    'O O O O TABELA O CONDICAO CONDICAO CONDICAO O O O ID O INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O O O TABELA O O ID O O INTERVALO INTERVALO?', 
    'O O TABELA CONDICAO O O O ID O O O O', 
    'O O O O O TABELA O O ID O O INTERVALO INTERVALO?', 
    'O O O TABELA O O ID O O O TABELA CONDICAO CONDICAO O INTERVALO INTERVALO?', 
    'O O O O O TABELA CONDICAO CONDICAO O O FORNECEDOR O INTERVALO INTERVALO INTERVALO?', 
    'O TABELA CONDICAO O O O O O ID O INTERVALO INTERVALO?', 
    'O O O O O TABELA O CONDICAO O TABELA O O ID O O INTERVALO INTERVALO INTERVALO?', 
    'O O O O O O FORNECEDOR O INTERVALO INTERVALO INTERVALO?', 
    'O O O O TABELA CONDICAO CONDICAO O INTERVALO INTERVALO INTERVALO INTERVALO O O ID?', 
    'O O O TABELA O O O TABELA CONDICAO O INTERVALO INTERVALO INTERVALO?', 
    'O O TABELA CONDICAO O O O ID O O INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O O O TABELA O CONDICAO CONDICAO CONDICAO O O ID O O INTERVALO INTERVALO INTERVALO INTERVALO INTERVALO INTERVALO?', 
    'O O TABELA CONDICAO CONDICAO O INTERVALO INTERVALO INTERVALO O O O O'
]

# Tokenização do texto e tags
X = tokenizer.texts_to_sequences(texts)
X = pad_sequences(X, padding='post')

# Tokenização das tags NER
tag_tokenizer = Tokenizer()
tag_tokenizer.fit_on_texts(ner_tags)
y = tag_tokenizer.texts_to_sequences(ner_tags)
y = pad_sequences(y, padding='post')
word2id_NER = tag_tokenizer.word_index
id2word_NER = {v: k for k, v in word2id_NER.items()}

# count = 0
# for sentense_x, sentense_y in zip(X, y):
#     if count == 2:   
#         word = [id2word[id] for id in sentense_x if id in id2word] 
#         ent = [id2word_NER[id] for id in sentense_y if id in id2word_NER] 
#         for i in range(len(word)):
#             print(word[i], ent[i])
#         print(len(word), len(ent))
#         break
#     count +=1



In [9]:
# Verificando as formas de X e y
print("Forma de X:", X.shape)
print("Forma de y:", y.shape)

Forma de X: (50, 20)
Forma de y: (50, 20)


In [10]:
# Definir o modelo NER
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
print(vocab_size)

131


In [11]:
input_layer = Input(shape=(X.shape[1],))
embedding_layer = Embedding(vocab_size, embedding_dim)(input_layer)
lstm_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)
output_layer = Dense(len(tag_tokenizer.word_index) + 1, activation='softmax')(lstm_layer)

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 20, 100)           13100     
                                                                 
 bidirectional (Bidirection  (None, 20, 128)           84480     
 al)                                                             
                                                                 
 dense_1 (Dense)             (None, 20, 9)             1161      
                                                                 
Total params: 98741 (385.71 KB)
Trainable params: 98741 (385.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
# Treinar o modelo
model.fit(X, y, epochs=1000, batch_size=64)

Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 2.1854 - accuracy: 0.1300
Epoch 2/1000
1/1 [==============================] - 0s 17ms/step - loss: 2.1660 - accuracy: 0.4920
Epoch 3/1000
1/1 [==============================] - 0s 14ms/step - loss: 2.1461 - accuracy: 0.5350
Epoch 4/1000
1/1 [==============================] - 0s 14ms/step - loss: 2.1254 - accuracy: 0.5350
Epoch 5/1000
1/1 [==============================] - 0s 14ms/step - loss: 2.1034 - accuracy: 0.5310
Epoch 6/1000
1/1 [==============================] - 0s 14ms/step - loss: 2.0797 - accuracy: 0.5470
Epoch 7/1000
1/1 [==============================] - 0s 14ms/step - loss: 2.0538 - accuracy: 0.5480
Epoch 8/1000
1/1 [==============================] - 0s 13ms/step - loss: 2.0251 - accuracy: 0.5510
Epoch 9/1000
1/1 [==============================] - 0s 15ms/step - loss: 1.9932 - accuracy: 0.5590
Epoch 10/1000
1/1 [==============================] - 0s 15ms/step - loss: 1.9573 - accuracy: 0.5610
Epoch 11/10

In [13]:
# Função para prever NER tags
def predict_ner(text):
    seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(seq, maxlen=X.shape[1], padding='post')
    pred = model.predict(padded_seq)[0]
    pred_tags = np.argmax(pred, axis=-1)
    return [tag_tokenizer.index_word[tag] for tag in pred_tags if tag != 0]

def ner_tags(text):
    tags = predict_ner(text)
    print(text)
    for sentense_x in tokenizer.texts_to_sequences([text]):   
            word = [id2word[id] for id in sentense_x if id in id2word]
            for w in range(len(word)):
                if not tags[w] == 'o':
                    print(f' - {word[w]}: {tags[w]}')
test_texts = [
    'Quais foram os fornecedores que mais venderam no último trimestre?',
    'Quantos pagamentos foram feitos manualmente em março deste ano?',
    'Qual a lista de fornecedores do grupo 2000?',
    'Como está o percentual de documentos vencidos este mês?',
    'Quem são os fornecedores com maior número de pagamentos em atraso?',
    'Quantos pagamentos foram efetuados após o vencimento no primeiro semestre de 2024?',
    'Qual é a relação de documentos a vencer em julho de 2024?'
]
# Exemplo de previsão
for text in test_texts:
    ner_tags(text)

1/1 [==============================] - 1s 567ms/step
Quais foram os fornecedores que mais venderam no último trimestre?
 - fornecedores: tabela
 - último: intervalo
 - trimestre: intervalo
1/1 [==============================] - 0s 49ms/step
Quantos pagamentos foram feitos manualmente em março deste ano?
 - pagamentos: tabela
 - manualmente: condicao
 - março: intervalo
 - ano: intervalo
1/1 [==============================] - 0s 48ms/step
Qual a lista de fornecedores do grupo 2000?
 - fornecedores: tabela
 - 2000: id
1/1 [==============================] - 0s 26ms/step
Como está o percentual de documentos vencidos este mês?
 - documentos: tabela
 - vencidos: condicao
 - mês: intervalo
1/1 [==============================] - 0s 28ms/step
Quem são os fornecedores com maior número de pagamentos em atraso?
 - fornecedores: tabela
 - pagamentos: tabela
1/1 [==============================] - 0s 28ms/step
Quantos pagamentos foram efetuados após o vencimento no primeiro semestre de 2024?
 - pagam